In [1]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:16.04

MAINTAINER Amazon AI <sage-learner@amazon.com>

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Install all of the packages
RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py
RUN pip install numpy
RUN pip install scipy
RUN pip install scikit-learn
RUN pip install pandas
RUN pip install flask
RUN pip install gevent
RUN pip install gunicorn
RUN pip install tensorflow
RUN pip install keras
RUN (cd /usr/local/lib/python2.7/dist-packages/scipy/.libs; rm *; ln ../../numpy/.libs/* .)
RUN rm -rf /root/.cache

# Env Variables
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${PATH}"

# Set up the program in the image


In [2]:
%%sh

# The name of our algorithm
algorithm_name=ann-churn

cd container

chmod +x ann/train
chmod +x ann/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/20 : FROM ubuntu:16.04
 ---> 0b1edfbffd27
Step 2/20 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 405adac98231
Step 3/20 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 4746cc5683b9
Step 4/20 : RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py
 ---> Using cache
 ---> 29cb6fc07171
Step 5/20 : RUN pip install numpy
 ---> Using cache
 ---> 5e2614189643
Step 6/20 : RUN pip install scipy
 ---> Using cache
 ---> 880699a8dfa3
Step 7/20 : RUN pip install scikit-learn
 ---> Using cache
 ---> 380b95e35c67
Step 8/20 : RUN pip install pandas
 ---> Using cache
 ---> 24d08829b386
Step 9/20 : RUN pip install flask
 ---> Using cache
 ---> f13d302d3292
Step 10/20 : RUN pip install gevent
 ---> Using cache
 ---> 1af7e1a6a554
Step 11/20 : RUN pip install gunicorn
 ---> Using cache
 ---> 6c

WARNING! Using --password via the CLI is insecure. Use --password-stdin.


## Testing your algorithm on your local machine or on an Amazon SageMaker notebook instance

While you're first packaging an algorithm use with Amazon SageMaker, you probably want to test it yourself to make sure it's working right. In the directory `container/local_test`, there is a framework for doing this. It includes three shell scripts for running and using the container and a directory structure that mimics the one outlined above.

The scripts are:

* `train_local.sh`: Run this with the name of the image and it will run training on the local tree. You'll want to modify the directory `test_dir/input/data/...` to be set up with the correct channels and data for your algorithm. Also, you'll want to modify the file `input/config/hyperparameters.json` to have the hyperparameter settings that you want to test (as strings).
* `serve_local.sh`: Run this with the name of the image once you've trained the model and it should serve the model. It will run and wait for requests. Simply use the keyboard interrupt to stop it.
* `predict.sh`: Run this with the name of a payload file and (optionally) the HTTP content type you want. The content type will default to `text/csv`. For example, you can run `$ ./predict.sh payload.csv text/csv`.

The directories as shipped are set up to test the decision trees sample algorithm presented here.

# Part 2: Training and Hosting your Algorithm in Amazon SageMaker

Once you have your container packaged, you can use it to train and serve models. Let's do that with the algorithm we made above.

## Set up the environment

Here we specify a bucket to use and the role that will be used for working with SageMaker.

In [3]:
# S3 prefix
prefix = 'ann-churn'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## Create the session

The session remembers our connection parameters to SageMaker. We'll use it to perform all of our SageMaker operations.

In [4]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using some the classic [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set), which we have included. 

We can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 

In [5]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

## Create an estimator and fit the model

In order to use SageMaker to fit our algorithm, we'll create an `Estimator` that defines how to use the container to train. This includes the configuration we need to invoke SageMaker training:

* The __container name__. This is constructed as in the shell commands above.
* The __role__. As defined above.
* The __instance count__ which is the number of machines to use for training.
* The __instance type__ which is the type of machine to use for training.
* The __output path__ determines where the model artifact will be written.
* The __session__ is the SageMaker session object that we defined above.

Then we use fit() on the estimator to train against the data that we uploaded above.

In [6]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{prefix}:latest'

classifier = sage.estimator.Estimator(
    image, 
    role, 
    1, 
    'ml.c4.2xlarge', 
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess)

classifier.fit(data_location)

INFO:sagemaker:Creating training-job with name: ann-churn-2018-05-27-18-29-21-010


.............
/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Starting the training.
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
Epoch 1/1

50833/50833 [==============================] - 3s 59us/step - loss: 0.1819 - acc: 0.9521
Epoch 1/1
50833/50833 [==============================] - 3s 59us/step - loss: 0.1722 - acc: 0.9602


Epoch 1/1

50834/50834 [==============================] - 3s 59us/step - loss: 0.2018 - acc: 0.9523
Epoch 1/1
50834/50834 [==============================] - 3s 60us/step - loss: 0.1857 - acc: 0.9607
Epoch 1/1

 3675/50834 [=>............................] - ETA: 6s - loss: 0.6567 - acc: 0.7298


50834/50834 [==============================] - 3s 61us/step - loss: 0.1933 - acc: 0.9511
Epoch 1/1

50834/50834 [==============================] - 3s 62us/step - loss: 0.2246 - acc: 0.9397
Epoch 1/1
50834/50834 [==============================] - 3s 63us/step - loss: 0.1958 - acc: 0.9571
Epoch 1/1

18875/50834 [==========>...................] - ETA: 2s - loss: 0.2875 - acc: 0.9358


50834/50834 [==============================] - 3s 63us/step - loss: 0.1825 - acc: 0.9605
Epoch 1/1

50834/50834 [==============================] - 3s 63us/step - loss: 0.1980 - acc: 0.9514
Epoch 1/1

25150/50834 [=============>................] - ETA: 1s - loss: 0.2710 - acc: 0.9220


...

50833/50833 [==============================] - 5s 98us/step - loss: 0.2318 - acc: 0.9453
Epoch 1/1
50833/50833 [==============================] - 5s 99us/step - loss: 0.2478 - acc: 0.9467
Epoch 1/1
50834/50834 [==============================] - 5s 99us/step - loss: 0.2117 - acc: 0.9550
Epoch 1/1


50834/50834 [==============================] - 5s 100us/step - loss: 0.2325 - acc: 0.9518
Epoch 1/1
50834/50834 [==============================] - 5s 101us/step - loss: 0.2197 - acc: 0.9521
Epoch 1/1
50834/50834 [==============================] - 5s 101us/step - loss: 0.2167 - acc: 0.9519


Epoch 1/1
50834/50834 [==============================] - 5s 102us/step - loss: 0.2143 - acc: 0.9554
Epoch 1/1
50834/50834 [==============================] - 5s 103us/step - loss: 0.2199 - acc: 0.9477
Epoch 1/1
25568/50834 [==============>...............] - ETA: 3s - loss: 0.3273 - acc: 0.9337


50834/50834 [==============================] - 5s 104us/step - loss: 0.2280 - acc: 0.9551
Epoch 1/1
50834/50834 [==============================] - 5s 107us/step - loss: 0.2441 - acc: 0.9486
Epoch 1/1
56482/56482 [==============================] - 7s 116us/step - loss: 0.1981 - acc: 0.9566
Training is complete.


===== Job Complete =====
Billable seconds: 560


## Deploy the model

Deploying the model to SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count, instance type, and optionally serializer and deserializer functions. These are used when the resulting predictor is created on the endpoint.

In [14]:
from sagemaker.predictor import csv_serializer
predictor = classifier.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

INFO:sagemaker:Creating model with name: ann-churn-2018-05-27-18-49-07-013
INFO:sagemaker:Creating endpoint with name ann-churn-2018-05-27-18-29-21-010


-------------------------------------------------------------!

## Choose some data and use it for a prediction

In order to do some predictions, we'll extract some of the data we used for training and do predictions against it. This is, of course, bad statistical practice, but a good way to see how the mechanism works.

In [15]:
# Importing the dataset
dataset = pd.read_csv('data/churn.csv')

for col in dataset.columns:
    if col not in ['user_id', 'domain_name']:
        col_mean = np.nanmean(dataset[col], axis=0)
        dataset[col].fillna(col_mean,inplace=True)
        
# Replace with average age
X = dataset.iloc[:, 1:17].values
y = dataset.iloc[:, 17].values

# Encoding categorical variables
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 15] = labelencoder_X_1.fit_transform(X[:, 15])

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

import itertools

a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

df = pd.DataFrame(X_test).iloc[indices[:-1]]

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Prediction is as easy as calling predict with the predictor we got back from deploy and the data we want to do predictions with. The serializers take care of doing the data conversions for us.

In [16]:
# # dict_test = {col: [val] for col, val in zip(dataset.columns, X_test[1])}
# # dict_test = pd.DataFrame.from_dict(dict_test, orient = 'index')
# # s = pd.Series(dict_test, name = 'test')
# df = pd.DataFrame(X_test[1])

# print(type(df))


print(predictor.predict(df.values).decode('utf-8'))


0.027434496
0.37452298
0.025588198
0.026960442
0.03179186
0.04434355
0.0278371
1.0
0.11551658
1.0
0.026616087
0.032466426
0.026545195
1.0
0.02555298
0.049248382
0.032999154
1.0
1.0
0.029439388
1.0
0.9999908
0.036826607
1.0
0.026471052
0.03185896
0.037169527
1.0
1.0



## Optional cleanup

When you're done with the endpoint, you'll want to clean it up.

In [17]:
sess.delete_endpoint(predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: ann-churn-2018-05-27-18-29-21-010
